# 1 - Captura de dados (Web Scrapping)

In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep


links = []
titles = []
for page in range(0,100,50):
    url = f'https://myanimelist.net/topanime.php?type=tv&limit={page}'

    # Send an HTTP GET request to the URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Now you can work with the soup object to extract data from the page
        # For example, you can find and print the title of the page:
        title = soup.title.string
        print(f"Title of the page: {title}")

        for element in soup.find_all('div', class_='di-ib clearfix'):
            links.append(element.find('a', class_="hoverinfo_trigger")['href'])
            titles.append(element.find('a', class_="hoverinfo_trigger").text)
    
    if page>0 and page%1000==0:
        sleep(310)
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")


Title of the page: 
  Top Anime - Top TV Series - MyAnimeList.net

Failed to retrieve the page. Status code: 200
Title of the page: 
  Top Anime - Top TV Series (50 - ) - MyAnimeList.net

Failed to retrieve the page. Status code: 200


In [2]:
import pandas as pd
anime_df = pd.DataFrame()
anime_df['Title'] = titles
anime_df['Link'] = links
anime_df

,Title,Link
0,Fullmetal Alchemist: Brotherhood,https://myanimelist.net/anime/5114/Fullmetal_A...
1,Sousou no Frieren,https://myanimelist.net/anime/52991/Sousou_no_...
2,Steins;Gate,https://myanimelist.net/anime/9253/Steins_Gate
3,Gintama°,https://myanimelist.net/anime/28977/Gintama°
4,Shingeki no Kyojin Season 3 Part 2,https://myanimelist.net/anime/38524/Shingeki_n...
...,...,...
95,Mob Psycho 100,https://myanimelist.net/anime/32182/Mob_Psycho...
96,Ansatsu Kyoushitsu 2nd Season,https://myanimelist.net/anime/30654/Ansatsu_Ky...
97,Uchuu Kyoudai,https://myanimelist.net/anime/12431/Uchuu_Kyoudai
98,Aria the Origination,https://myanimelist.net/anime/3297/Aria_the_Or...


In [3]:
anime_df = pd.DataFrame()

j = 1
for row, link in enumerate(links):
    url = link
    # Send an HTTP GET request to the URL
    response = requests.get(url)
    sinopsis  =  []
    score =      []
    episodes =   []
    img_link =   []
    status =     []
    aired =      []
    producers =  []
    licensors =  []
    studios =    []
    source =     []
    genres =     []
    themes =     []
    rating =     []
    popularity = []
    members =    []
    favorites =  []
    demography = []
    #adaptation = []
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Now you can work with the soup object to extract data from the page
        # For example, you can find and print the title of the page:
        title = soup.title.string
        print(f"Title of the page: {title}", f" remaning {len(links)-j}")
        
        sinopsis.append(soup.find('p', itemprop="description").text)
        img_link.append(soup.find('div',  style="text-align: center;").find('a').find('img', class_='ac')['data-src'])
        #adaptation.append(soup.find('td', class_="borderClass", width='100%').get_text())
        score.append(soup.find('span', class_="score-label").get_text(strip=True))        
        info_list = [] #Será como o data_split na 1a versão/tentativa do código
        info_list.append(soup.find_all('div', class_="spaceit_pad"))        
        info_list = info_list[0]
        for i in info_list:
            if i.find('span', itemprop = "genre") and (i.find('span', class_="dark_text").text=='Genres:') | (i.find('span', class_="dark_text").text=='Genre:') :
                # Find all <a> elements within the <div> that have specific attributes
                genre_elements = i.find_all('a', {'title': True, 'href': True})
                all_genres = [element.get_text(strip=True) for element in genre_elements]
                if len(all_genres)>0:
                    genres.append( all_genres)
                else:
                    genres.append('None')
                
            if i.find('span', itemprop = "genre") and (i.find('span', class_="dark_text").text=='Themes:') | (i.find('span', class_="dark_text").text=='Theme:'):
                theme_elements = i.find_all('a', {'title': True, 'href': True})
                all_themes = [element.get_text(strip=True) for element in theme_elements]
                if len(all_themes)>0:
                    themes.append( all_themes)
                else:
                    themes.append('None')        
                    
            if i.find('span', itemprop = "genre") and (i.find('span', class_="dark_text").text=='Demographics:') | (i.find('span', class_="dark_text").text=='Demographic:'):
                demo_elements = i.find_all('a', {'title': True, 'href': True})
                all_demos = [element.get_text(strip=True) for element in demo_elements]
                if len(all_themes)>0:
                    demography.append( all_demos)
                else:
                    demography.append('None')        
        t = []
        for i in info_list:
            i = i.text.replace('\n',' ').strip()#.replace('   ', ' ')
            t.append(i)
        
        #demography.append([s for s in t if "Demographic:" in s][0].replace('Demographic:', '').replace(' ', ''))
        episodes.append([s for s in t if "Episodes:" in s][0].replace('Episodes:', '').replace(' ', ''))
        status.append([s for s in t if "Status:" in s][0].replace('Status:', ''))
        aired.append([s for s in t if "Aired:" in s][0].replace('Aired:  ', '').split(' to ')[0])
        producers.append([s for s in t if "Producers:" in s][0].replace('Producers:', '').replace('  ', ''))
        licensors.append([s for s in t if "Licensors:" in s][0].replace('Licensors:', '').replace('  ', ''))
        studios.append([s for s in t if "Studios:" in s][0].replace('Studios:', '').replace('  ', ''))
        source.append([s for s in t if "Source:" in s][0].replace('Source:', '').replace('  ', ''))
        rating.append([s for s in t if "Rating:" in s][0].replace('Rating:', '').replace('  ', ''))
        popularity.append([s for s in t if "Popularity:" in s][0].replace('Popularity:', '').replace('  ', ''))
        members.append([s for s in t if "Members:" in s][0].replace('Members:', '').replace('  ', ''))
        favorites.append([s for s in t if "Favorites:" in s][0].replace('Favorites:', '').replace('  ', ''))
        anime = soup.find('h1', class_='title-name h1_bold_none').text
        anime_df[anime] = pd.Series([sinopsis[0], 
                                     img_link[0], 
                                     demography, 
                                     score[0], 
                                     genres, 
                                     themes, 
                                     episodes[0], 
                                     status[0], 
                                     aired[0], 
                                     producers[0],
                                     licensors[0],
                                     studios[0],
                                     source[0], 
                                     rating[0], 
                                     popularity[0],
                                     members[0], 
                                     favorites[0]]).T
        #print(anime_df)
        j+=1
        if j%110==0:
            sleep(330)
    else:
        print(f"Failed to retrieve the page {title}. Status code: {response.status_code}")


Title of the page: 
Fullmetal Alchemist: Brotherhood - MyAnimeList.net
  remaning 99
Title of the page: 
Sousou no Frieren (Frieren: Beyond Journey's End) - MyAnimeList.net
  remaning 98
Title of the page: 
Steins;Gate - MyAnimeList.net
  remaning 97
Title of the page: 
Gintama° (Gintama Season 4) - MyAnimeList.net
  remaning 96
Title of the page: 
Shingeki no Kyojin Season 3 Part 2 - MyAnimeList.net
  remaning 95
Title of the page: 
Bleach: Sennen Kessen-hen (Bleach: Thousand-Year Blood War) - MyAnimeList.net
  remaning 94
Title of the page: 
Hunter x Hunter (2011) (Hunter x Hunter) - MyAnimeList.net
  remaning 93
Title of the page: 
Gintama' (Gintama Season 2) - MyAnimeList.net
  remaning 92
Title of the page: 
Gintama': Enchousen (Gintama: Enchousen) - MyAnimeList.net
  remaning 91
Title of the page: 
Kaguya-sama wa Kokurasetai: Ultra Romantic - MyAnimeList.net
  remaning 90
Title of the page: 
Fruits Basket: The Final (Fruits Basket: The Final Season) - MyAnimeList.net
  remaning 8

# 2 - Preparação dos Dados (Data organization)

In [4]:

anime_df.index=['sinopsis', 'img_link', 'demographics', 'score',
                               'genres', 'themes', 'episodes', 'status', 'aired', 'producers',
                                 'licensors', 'studios','source', 'rating', 'popularity', 'members', 'favorites']

In [5]:
###### Formatando Data

from datetime import datetime


# Convert the Series to datetime with the desired format
# Remove leading spaces from the date strings
date_series = anime_df.loc['aired'].str.strip()

# Initialize an empty list for the result
formatted_dates = []

# Process each date string in the list
for date_string in date_series:
    try:
        # Try to parse the date string with a specific format
        date = datetime.strptime(date_string, '%b %d, %Y')
        formatted_dates.append(date.strftime('%Y-%m-%d'))
    except ValueError:
        try:
            # Try to parse the date string with another format (e.g., '2009')
            date = datetime.strptime(date_string, '%Y')
            formatted_dates.append(date.strftime('%Y'))
        except ValueError:
            try:
                # Try to parse the date string with yet another format (e.g., ' Oct 1999')
                date = datetime.strptime(date_string, '%b %Y')
                formatted_dates.append(date.strftime('%Y-%m'))
            except ValueError:
                # Handle cases where the date format is not recognized
                formatted_dates.append('Invalid Date')

# Print the formatted dates
print(formatted_dates)


['2009-04-05', '2023-09-29', '2011-04-06', '2015-04-08', '2019-04-29', '2022-10-11', '2011-10-02', '2011-04-04', '2012-10-04', '2022-04-09', '2021-04-06', '2017-01-09', '2006-04-04', '2017-10-14', '2008-10-03', '2008-04-06', '2018-07-09', '2023-07-06', '2017-08-12', '2004-04-07', '2023-01-10', '2020-04-06', '2022-10-09', '2018-01-08', '2019-01-07', '2020-12-07', '2021-12-05', '2016-10-08', '2000-10-04', '2022-01-10', '2013-07-07', '2019-07-08', '1998-04-03', '2023-04-12', '2022-04-10', '2014-10-19', '2023-07-08', '1980-10-13', '2017-01-07', '2022-10-06', '1999-10-20', '2021-10-03', '2006-10-06', '2014-04-05', '2021-10-04', '1999-06-30', '2023-10-22', '2021-04-06', '2023-07-12', '2018-01-11', '2017-07-07', '2005-10-23', '2009-01-07', '2022-07-06', '2014-10-10', '2012-01-03', '2020-04-11', '2015-10-04', '2020-10-03', '2007-04-01', '2006-10-04', '2023-10-04', '2017-04-12', '2014-04-11', '2018-07-23', '2013-10-06', '1997-10-08', '2018-10-06', '2016-10-05', '2011-07-05', '2016-01-09', '2022

In [6]:
import numpy as np 

clean_anime_df = anime_df.copy()

clean_anime_df.loc['aired'] = formatted_dates

clean_genres = []
for i in clean_anime_df.loc['genres']:
    if len(i)==1:
        i=i[0]
    else:
        i= ', '.join(i)
    clean_genres.append(i)
clean_anime_df.loc['genres'] = np.array(clean_genres, dtype=object)

clean_themes = []
for i in clean_anime_df.loc['themes']:
    if len(i)==0:
        i='None'
    else:
        i= ', '.join(i[0])
    #print(i)
    clean_themes.append(i)
clean_anime_df.loc['themes'] = clean_themes

clean_demos = []
for i in clean_anime_df.loc['demographics']:
    if len(i)==1:
        i=i[0]
    else:
        i= ', '.join(i)
    clean_demos.append(i)
clean_anime_df.loc['demographics'] = np.array(clean_demos, dtype=object)

clean_popularity = []
for i in clean_anime_df.loc['popularity']:
    
    i = int(i.replace(' ','').replace('#', ''))
    #print(i)
    clean_popularity.append(i)
clean_anime_df.loc['popularity'] =  np.array(clean_popularity, dtype=object)


clean_members = []
for i in clean_anime_df.loc['members']:
    
    i = int(i.replace(' ','').replace(',', ''))
    #print(i)
    clean_members.append(i)
clean_anime_df.loc['members'] =  np.array(clean_members, dtype=object)


clean_favorites = []
for i in clean_anime_df.loc['favorites']:
    
    i = int(i.replace(' ','').replace(',', ''))
    #print(i)
    clean_favorites.append(i)
clean_anime_df.loc['favorites'] =  np.array(clean_favorites, dtype=object)



In [7]:
clean_anime_df.loc['links'] = links
clean_anime_df

,Fullmetal Alchemist: Brotherhood,Sousou no Frieren,Steins;Gate,Gintama°,Shingeki no Kyojin Season 3 Part 2,Bleach: Sennen Kessen-hen,Hunter x Hunter (2011),Gintama',Gintama': Enchousen,Kaguya-sama wa Kokurasetai: Ultra Romantic,...,JoJo no Kimyou na Bouken Part 4: Diamond wa Kudakenai,Summertime Render,Yakusoku no Neverland,One Punch Man,Kimetsu no Yaiba,Mob Psycho 100,Ansatsu Kyoushitsu 2nd Season,Uchuu Kyoudai,Aria the Origination,Banana Fish
sinopsis,After a horrific alchemy experiment goes wrong...,During their decade-long quest to defeat the D...,Eccentric scientist Rintarou Okabe has a never...,"Gintoki, Shinpachi, and Kagura return as the f...",Seeking to restore humanity's diminishing hope...,Substitute Soul Reaper Ichigo Kurosaki spends ...,Hunters devote themselves to accomplishing haz...,"After a one-year hiatus, Shinpachi Shimura ret...","While Gintoki Sakata was away, the Yorozuya fo...",The elite members of Shuchiin Academy's studen...,...,"The year is 1999. Morioh, a normally quiet and...","Since the death of his parents, Shinpei Ajiro ...","Surrounded by a forest and a gated entrance, t...",The seemingly unimpressive Saitama has a rathe...,"Ever since the death of his father, the burden...","Eighth-grader Shigeo ""Mob"" Kageyama has tapped...","Returning from their summer vacation, the stud...","On a fateful summer night in 2006, Mutta Nanba...","In the 24th century on the planet Aqua, three ...","Aslan Jade Callenreese, known as Ash Lynx, was..."
img_link,https://cdn.myanimelist.net/images/anime/1208/...,https://cdn.myanimelist.net/images/anime/1015/...,https://cdn.myanimelist.net/images/anime/1935/...,https://cdn.myanimelist.net/images/anime/3/720...,https://cdn.myanimelist.net/images/anime/1517/...,https://cdn.myanimelist.net/images/anime/1908/...,https://cdn.myanimelist.net/images/anime/1337/...,https://cdn.myanimelist.net/images/anime/4/503...,https://cdn.myanimelist.net/images/anime/1452/...,https://cdn.myanimelist.net/images/anime/1160/...,...,https://cdn.myanimelist.net/images/anime/3/791...,https://cdn.myanimelist.net/images/anime/1120/...,https://cdn.myanimelist.net/images/anime/1830/...,https://cdn.myanimelist.net/images/anime/12/76...,https://cdn.myanimelist.net/images/anime/1286/...,https://cdn.myanimelist.net/images/anime/8/803...,https://cdn.myanimelist.net/images/anime/8/779...,https://cdn.myanimelist.net/images/anime/1290/...,https://cdn.myanimelist.net/images/anime/1394/...,https://cdn.myanimelist.net/images/anime/1190/...
demographics,[Shounen],[Shounen],,[Shounen],[Shounen],[Shounen],[Shounen],[Shounen],[Shounen],[Seinen],...,[Shounen],[Shounen],[Shounen],[Seinen],[Shounen],,[Shounen],[Seinen],[Shounen],[Shoujo]
score,9.10,9.09,9.07,9.06,9.05,9.04,9.04,9.03,9.03,9.03,...,8.50,8.50,8.50,8.50,8.49,8.49,8.49,8.49,8.48,8.48
genres,"[Action, Adventure, Drama, Fantasy]","[Adventure, Drama, Fantasy]","[Drama, Sci-Fi, Suspense]","[Action, Comedy, Sci-Fi]","[Action, Drama, Suspense]","[Action, Adventure, Fantasy]","[Action, Adventure, Fantasy]","[Action, Comedy, Sci-Fi]","[Action, Comedy, Sci-Fi]","[Comedy, Romance]",...,"[Action, Adventure, Supernatural]","[Mystery, Supernatural, Suspense]","[Fantasy, Mystery, Sci-Fi, Suspense]","[Action, Comedy]","[Action, Award Winning, Fantasy]","[Action, Comedy, Supernatural]","[Action, Comedy]","[Comedy, Sci-Fi]","[Fantasy, Sci-Fi, Slice of Life]","[Action, Adventure, Drama, Suspense]"
themes,Military,None,"Psychological, Time Travel","Gag Humor, Historical, Parody, Samurai","Gore, Military, Survival",None,None,"Gag Humor, Historical, Parody, Samurai","Gag Humor, Historical, Parody, Samurai",School,...,None,Time Travel,"Psychological, Survival","Adult Cast, Parody, Super Power",Historical,Super Power,School,"Adult Cast, Space, Workplace",Iyashikei,"Delinquents, Organized Crime, Psychological"
episodes,64,28,24,51,10,13,148,51,13,13,...,39,25,12,12,26,12,25,99,13,24
status,Finished Airing,Currently Airing,Finished Airing,Finished Airing,Finished Airing,Finished

In [8]:
anime_df.to_csv('MAL_data.csv')